In [4]:
import camb
import numpy as np
from scipy import special
import matplotlib.pyplot as plt
import sys, os, time
from szlib.szcounts import ClusterCosmology,SZ_Cluster_Model,Halo_MF,dictFromSection

from orphics.tools.output import Plotter
from ConfigParser import SafeConfigParser

beam = 1.5
noise = 1.0

iniFile = "../input/cosmology.ini"
Config = SafeConfigParser()
Config.optionxform=str
Config.read(iniFile)

cosmoDict = dictFromSection(Config,'WMAP9')
constDict = dictFromSection(Config,'constants')
cc = ClusterCosmology(cosmoDict,constDict)

SZProfExample = SZ_Cluster_Model(clusterCosmology=cc,spec_file='../input/test_scalCls_new.dat',rms_noise = noise,fwhm=beam,M=5e14,z=0.5)

TCPAR = 25.

def y2D_tilde_norm(ell,thtc,thta):                                                                                                                                                                                    
        ans = ell*0.
        y2D_use = SZProfExample.y2D_norm(thta/thtc)
        for ii in xrange(len(ell)):
            ans[ii] = np.sum(thta*special.jv(0,ell[ii]*thta)*y2D_use)*SZProfExample.dtht
        return ans, y2D_use

DAz = SZProfExample.cc.results.angular_diameter_distance(0.5) * 0.7

thtc = SZProfExample.R500/DAz

for ii in xrange(5):
    thta = np.arange(SZProfExample.dtht,TCPAR/(ii+1)*thtc,SZProfExample.dtht)    

    ytild,y2d = y2D_tilde_norm(SZProfExample.ells,thtc,thta)

    var = np.sum(SZProfExample.ells*ytild**2/SZProfExample.nl)*SZProfExample.dell*SZProfExample.freq_fac
    prof_int = 2.*np.pi*(np.sum((y2d*thta)[thta < thtc*5.])*SZProfExample.dtht)**2

    print np.sqrt(prof_int/var)


8.54943202791e-13
8.5494318967e-13
8.54944456875e-13
8.54937909358e-13
8.54917422994e-13
